In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np

In [ ]:
# Configurando o estilo de exibição de fl
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# Carregando e lendo o dataset
sales = pd.read_csv('C:/Users/mathe/Projetos Dados/EcommerceSalesAnalysis/App/Data/Online Retail.csv', delimiter=';', encoding='latin1', decimal=',')
sales.head()

#### Deve ser adicionada uma coluna de Preço Final de venda/devolução para cada transação

In [ ]:
sales.shape

#### Uma análise do Dataset mostrou que além de registrar as operações de venda e cancelamento dos produtos, esse dataset também registra operações manuais de ajustes, descontos ofertados, taxas, comissões e outras operações.

In [ ]:
# Filtrando as operações que não se referem a produtos
def not_number(value):
    return not value[0].isdigit()

# Filtra todos os registros que não vendas/cancelamento de produtos
not_product_lines = sales[sales['StockCode'].apply(not_number)]

# Obter os valores únicos das linhas
not_product = not_product_lines['StockCode'].unique()

# Imprimir os valores únicos que não começam com números
print(not_product)

#### Portanto, como a análise está focada apenas na VENDA e CANCELAMENTO dos PRODUTOS, os registros citados devem ser excluídos no pré-processamento.

#### Essas operações são identificadas pela coluna *[StockCode]*, em que seus valores iniciam-se por uma letra. As linhas que contem os dados presentes em "not_product", conforme mostrado acima, devem ser excluídas, por não se tratarem de vendas/cancelamentos de produtos.

In [ ]:
# Conhecendo os tipos dos dados
sales.dtypes

#### Pela análise dos tipos dos dados desse dataset, devem ser aplicadas transformações nos tipos de dados durante o pré-processamento.

In [ ]:
# Resumo dos dados
sales[['Quantity', 'UnitPrice']].describe().T

#### A análise das estatísticas descritivas dos dados mostra o valor mínimo de *[UnitPrice]* sendo negativo. Portanto, devem ser excluídos no pré-processamento os registros com valor unitário menor ou igual a zero, pois não configuram-se como vendas ou devoluções. 
#### Provavelmente são decorrentes de erros de registro

In [ ]:
# Verifica a presença de registros duplicados no dataset
cols_list = sales.columns
duplicates = sales.duplicated(subset=cols_list, keep=False)
sales[duplicates].sort_values('Description').head()

#### Como foi notada a presença de duplicatas no dataset, será necessário removê-las durante o pré-processamento

In [ ]:
# Checando valores ausentes
sales.isna().sum()

#### Há presença de valores ausentes na coluna *[CustomerID].* e na coluna *[Description]*

#### Esses valores nulos devem ser tratados de acordo com a análise realizada, principalmente para a coluna *[CustomerID]*, que possui uma grande quantidade de valores nulos.

#### Como essa quantidade é significativa, as linhas com valores nulos de *[CustomerID]* só serão removidas durante a análise dos clientes, mas serão mantidas nas análises de produtos, países e sazonalidade, já que a remoção iria afetar profundamente essas análises.

In [ ]:
# Verificando a existência de nomes de produtos iguais para diferentes StockCodes
duplicates_descriptions = sales.groupby('Description')['StockCode'].nunique()

# Filtra os casos onde há mais de um StockCode para a mesma Description
duplicates_descriptions = duplicates_descriptions[duplicates_descriptions > 1]

# Imprime as Description com mais de um StockCode associado
print("Descriptions com mais de um StockCode associado:")
print(duplicates_descriptions.sort_values(ascending=False))

In [ ]:
# Verificando a existência de StockCodes iguais para diferentes Description
duplicates_stock_codes = sales.groupby('StockCode')['Description'].nunique()

# Filtra os casos onde há mais de um Description para um mesmo StockCode
duplicates_stock_codes = duplicates_stock_codes[duplicates_stock_codes > 1]

# Imprime os StockCodes com mais de um Description associado
print("StockCodes com mais de um Description associado:")
print(duplicates_stock_codes.sort_values(ascending=False))

#### Há dois casos observados:
#### 1. Um mesmo código possui descrições diferentes
#### 2. Códigos diferentes possuem descrições iguais.

#### Para a análise feita neste dataset, será considerado que esses casos são decorrentes de cadastros errados, duplicados, registros incorretos ou alterações propositais nas descrições. 

In [ ]:
# Quantidade de valores únicos da coluna Description
sales.nunique()

#### Para simplificar a análise e melhorar a identificação dos produtos, cada registro único de *[Description]* será considerado como um produto diferente, já que a quantidade valores únicos de StockCode e Description são bem próximos.